In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from keras.models import load_model
import cv2
import os
import numpy as np
from pygame import mixer
import time


pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mixer.init()
sound = mixer.Sound('alarm.wav')
    
face = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml')

In [3]:
model = load_model('models/cnnAction.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

In [5]:
while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2] 
    #print(frame.shape) #(480, 640, 3)
    faces = face.detectMultiScale(frame,minNeighbors=5,scaleFactor=1.1)
    left_eye = leye.detectMultiScale(frame)
    right_eye =  reye.detectMultiScale(frame)
    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye= r_eye.reshape(224,224,3)
        r_eye=img_to_array(r_eye)
        r_eye=preprocess_input(r_eye)
        r_eye=np.array(r_eye,dtype='float32')

        rpred = model.predict(r_eye)
        rpred=np.argmax(rpred,axis=1)
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break
    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye= l_eye.reshape(224,224,3)
        l_eye=img_to_array(l_eye)
        l_eye=preprocess_input(l_eye)
        #l_eye= l_eye.reshape(224,224,3)
        l_eye=np.array(l_eye,dtype='float32')
        
        lpred = model.predict(l_eye)
        lpred=np.argmax(lpred,axis=1)
        if(lpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break
    if(rpred[0]==0 and lpred[0]==0):
            score=score+1
            cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)


    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>15):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()

        except:  # isplaying = False
            pass
        if(thicc<16):
                thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



ValueError: cannot reshape array of size 9075 into shape (224,224,3)